<center>
    <tr>
    <td><img src="images/Quansight_Logo_Lockup_1.png" width="25%"></img></td>
    </tr>
</center>

---
# Logistic Regression with PyTorch

---
## Outline

1. Using PyTorch data utilities
2. Constructing Logistic Regression model in PyTorch
3. Training the neural network

+ Objective: use logistic regression to solve binary classification problem to examplify workflow with PyTorch
  + Framework extends to training much larger deep network models

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pprint as pp
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn.parameter import Parameter
from sklearn import datasets

In [ ]:
np.random.seed(0)

+ Generate simple data: linear binary classification problem in 2D
+ Convenient `make_blobs` utility in Scikit-Learn useful here

In [ ]:
n_samples = 120
x, y = datasets.make_blobs(n_samples=n_samples, n_features=2, centers=[(-1,-1),(1.4,1)], cluster_std=[.2,.4], random_state=0)

plt.figure(figsize=(7,7))
plt.scatter(x[:,0], x[:,1], c=y, cmap=cm.bwr)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.xlim(-3,3)
plt.ylim(-3,3);

+ `make_moons` generates more sophisticated geometry
+ Linear binary classifier should fail

In [ ]:
n_samples = 120
x, y = datasets.make_moons(n_samples=n_samples, random_state=0, noise=0.1)
x = x - np.mean(x,0) # 0 centered

plt.figure(figsize=(7,7))
plt.scatter(x[:,0], x[:,1], c=y, cmap=cm.bwr)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.xlim(-3,3)
plt.ylim(-3,3)
plt.title(f'#data = {n_samples}');

---
## Using PyTorch data utilities

+ Deep learning models data intensive
   + Organizing data to support training deep neural networks time-consuming

#### PyTorch `Dataset` & `DataLoader` classes

+ [PyTorch `Dataset` class](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) (in `torch.utils.data`) for constructing appropriate _data loaders_ for deep network training
+ Abstraction generalizes to work with large on-disk data sets
+ [PyTorch `DataLoader` class](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) (again, in `torch.utils.data`) combines dataset and sampling strategy (e.g., in random batches without replacement) as Python iterable

In [ ]:
from torch.utils.data import Dataset
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        sample = {
            'feature': torch.tensor(self.x[idx], dtype=torch.float32), 
            'label': torch.tensor(np.array([self.y[idx]]), dtype=torch.float32)}
        return sample

In [ ]:
import pprint as pp

dataset = MyDataset(x, y)
print('length: ', len(dataset))
for i in range(5):
    pp.pprint(dataset[i])

+ Construct a `DataLoader` for batches in training

In [ ]:
from torch.utils.data import DataLoader

dataset = MyDataset(x, y) # Instantiate dataset
batch_size = 4
shuffle = True
num_workers = 4
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

In [ ]:
for i_batch, samples in enumerate(dataloader):
    if i_batch % 10 == 0:
        print('\nbatch# = %s' % i_batch)
        print('samples: ')
        pp.pprint(samples)

---
## Constructing Logisitic Regression model in PyTorch

+ With data ready, next construct PyTorch model
+ Inherit model class from PyTorch `nn.Module` class
   + Needs to provide `forward` method

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegression, self).__init__()
        
        output_dim = 1
        self.weight = Parameter(torch.Tensor(input_dim, 1))
        self.bias = Parameter(torch.Tensor(1, 1))
        
        stdv = 1.
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)
        
    def forward(self, x):
        a = torch.mm(x, self.weight)
        b = torch.mul(torch.ones(x.size(0),1), self.bias)
        c = torch.add(a, b)
        d = torch.sigmoid(c)#torch.exp(-c)
        return d#1./(1. + d)

#### Defining a Loss Function

+ Can be defined using class `nn.Module` again
+ When instantiated, method `MyLoss.forward(predictions, target)` computes *binary cross-entropy*
  $$ \mathcal{L}(\hat{y}, y) = -\sum_{k=1}^{N} \left[ y_{k} \log\left( \hat{y}_{k} \right) + \left(1-y_{k}\right) \log\left(1- \hat{y}_{k} \right) \right]$$
+ Typical loss function for *classification* problems

In [ ]:
class MyLoss(nn.Module):
    def __init__(self):
        super(MyLoss, self).__init__()
        
    def forward(self, predictions, targets):
        log_probs = torch.where(targets.byte(), torch.log(predictions), torch.log(1.-predictions))
        loss = - torch.sum(log_probs)
        return loss

#### Computing accuracy

+ Useful for classification problems

In [ ]:
def accuracy(predictions, targets):
    p = (predictions > 0.5)
    s = torch.sum(p.eq(targets.bool()))
    return s.item()

---
## Training the neural network

+ Module [`torch.optim`](https://pytorch.org/docs/stable/optim.html) supports numerous oprimization algorithms
  + Custom strategies can be defined (e.g., [*stochastic gradient descent*](https://en.wikipedia.org/wiki/Stochastic_gradient_descent), [ADAM](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Adam), [AdaGrad](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#AdaGrad), etc.)
+ Here, define `optimizer` using `torch.optim.SGD`


In [ ]:
import torch.nn.functional as F

model = LogisticRegression(2)
criterion = MyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  

dataset = MyDataset(x, y)
batch_size = 16
shuffle = True
num_workers = 4
training_sample_generator = DataLoader(dataset, 
                                       batch_size=batch_size, 
                                       shuffle=shuffle, 
                                       num_workers=num_workers)

num_epochs = 50
for epoch in range(num_epochs):
    n = 0
    for batch_i, samples in enumerate(training_sample_generator):
        predictions = model(samples['feature'])
        error = criterion(predictions, samples['label'])
        n += accuracy(predictions, samples['label'])
        optimizer.zero_grad()
        error.backward()        
        optimizer.step()

    if epoch % 10 == 0:
        print(f'epoch={epoch:03}. error={error.item():<7.4}. accuracy={n}')
    
    # If we have achieved 99% accuracy, then stop.
    if n > .99 * n_samples: 
        break
print(f'epoch={epoch:03}. error={error.item():<7.4}. accuracy={n}')

## Results

Colors represent whether or not points are classified correctly.

In [ ]:
predicted_labels = np.zeros(n_samples)
prob_of_one = model(torch.Tensor(x)).detach().numpy().flatten()
predicted_labels[prob_of_one > 0.5] = 1

# Color 1 represent correct classification, 0 otherwise
colors = np.where(predicted_labels == y, 1, 0) 

acc = np.sum(colors)
print(f'accuracy = {acc/len(colors)}')

In [ ]:
plt.figure(figsize=(7,7))
plt.title('Classification results')
plt.scatter(x[:,0], x[:,1], c=colors, cmap=cm.bwr)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.xlim(-3,3)
plt.ylim(-3,3);

## Visualizing classification results

In [ ]:
xcoord = np.linspace(-3, 3)
ycoord = np.linspace(-3, 3)
xx, yy = np.meshgrid(xcoord, ycoord)
xxt = torch.tensor(xx, dtype=torch.float32).view(-1).unsqueeze(0)
yyt = torch.tensor(yy, dtype=torch.float32).view(-1).unsqueeze(0)
v = torch.t(torch.cat([xxt,yyt]))
m = model(v)
mm = m.detach().numpy().reshape(50,50)

x_try = torch.tensor(x, dtype=torch.float32)
y_try = model(x_try)
yy_try = (y_try.squeeze() > 0.5).numpy()

plt.figure(figsize=(7,7))
extent = -3, 3, -3, 3
plt.imshow(mm, cmap=cm.BuGn, interpolation='bilinear', extent=extent, alpha=.5, origin='lower')
plt.scatter(x[:,0], x[:,1], c=yy_try, cmap=cm.viridis)
plt.colorbar()
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.xlim(-3,3)
plt.ylim(-3,3)
plt.title('Classification results');

## Summary

+ Using PyTorch data utilities
+ Constructing Logistic Regression model in PyTorch
+ Training the neural network
+ Framework extends to training much larger deep network models

<center>
    <tr>
    <td><img src="images/Quansight_Logo_Lockup_1.png" width="25%"></img></td>
    </tr>
</center>